<a href="https://colab.research.google.com/github/whorseman/Assignments/blob/main/homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following assignment consists again of a theoretical part (learning portfolio) and a practical part (assignment). The goal is to train a neural model from scratch, as we did a few weeks ago. I would like us to repeat the most important things so that we can consolidate our knowledge in this area, for this you will train a model on a dataset that I will provide via OLAT. 

The plan would be that in the first we will discuss your learnings from the theory part, that means you are relatively free to fill your Learning Portfolio on this topic and in the following week we will discuss your solutions of the Classification Model.

#Theory part (filling your Learning Portfolio, May 24)

In preparation for the practical part, I ask you to familiarize yourself with the following video sources in the next week:

1) Please watch the following video until random forests:

https://course.fast.ai/Lessons/lesson5.html

2) Please download the following notebooks and edit it in Google-Colab. Take notes and update your Learning Portfolio.

https://www.kaggle.com/code/jhoward/linear-model-and-neural-net-from-scratch

https://www.kaggle.com/code/jhoward/why-you-should-use-a-framework

The contents should be mostly known to you, because these contents are also based on chapter 4, which we have already worked on. We repeat the basics again with a new data set to consolidate our knowledge.



#Practical part (Assignment, May 31)

The following task is a binary classification task. The first column is our target. All variables are categorical variables from which you have to create dummy variables. The target column has only the property Q or F, which is to be modeled. As always, please create a train and a test data set, e.g. 80:20 or 90:10. Your goal is to create a neural network that best predicts the target column. Use probabilities with the sigmoid function as discussed in the theory. When designing your neural network, play with a few different numbers and sizes of layers and different activation functions.

To validate your model, you can compare your results with a logistic regression model. (Note: This data set should allow you to achieve very high accuracies: > 98% maybe even >99%) You can see how these results vary when you use 50% for testing and 50% for training.

In [ ]:
import pandas as pd
from google.colab import drive
from google.colab import data_table
import matplotlib.pyplot as plt
data_table.enable_dataframe_formatter()

drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/Python/ML-Course/6/data_homework_6.csv")

data.head(10)

When we have discussed the assignment, I will tell you what the data set is about, which has its origin from our nature.